In [ ]:
import distributed
import dask
import pystac_client
from dask.distributed import Client
import ipyleaflet
import planetary_computer as pc
import IPython.display as dsp
import geogif
import stackstac

In [ ]:
# Paste /proxy/localhost:8787 for cluster diagnostics
client = Client(local_directory='/tmp', processes=False)

In [ ]:
m = ipyleaflet.Map(scroll_wheel_zoom=True)
m.center = 39.03053, -108.0322
m.zoom = 12
m.layout.height = "800px"
m

In [ ]:
bbox = (m.west, m.south, m.east, m.north)

In [ ]:
URL = "https://earth-search.aws.element84.com/v0"
catalog = pystac_client.Client.open(URL)

search = catalog.search(
    collections=["sentinel-s2-l2a-cogs"],
    bbox=bbox,
)

In [ ]:
%%time
items = pc.sign(search)
len(items)

In [ ]:
dsp.GeoJSON(items.to_dict())

In [ ]:
%%time
epsg_code=32612
stack = stackstac.stack(items, epsg=epsg_code, bounds_latlon=bbox)
stack

In [ ]:
# use common_name for bands
stack = stack.assign_coords(band=stack.common_name.fillna(stack.band).rename("band"))
stack.band

In [ ]:
stack.sel(band=["red", "green", "blue"])

In [ ]:
# Make a bitmask---when we bitwise-and it with the data, it leaves just the 4 bits we care about
mask_bitfields = [1, 2, 3, 4]  # dilated cloud, cirrus, cloud, cloud shadow
bitmask = 0
for field in mask_bitfields:
    bitmask |= 1 << field

bin(bitmask)

In [ ]:
qa = stack.sel(band="overview")
#bad = qa & bitmask  # just look at those 4 bits

#good = stack.where(bad == 0)  # mask pixels where any one of those bits are set

In [ ]:
good=stack

In [ ]:
# What's the typical interval between scenes?
good.time.diff("time").dt.days.plot.hist();

In [ ]:
# Make biannual median composites (`2Q` means 2 quarters)
composites = good#.resample(time="2Q").median("time")
composites

In [ ]:
rgb = composites.sel(band=["red", "green", "blue"])
rgb

In [ ]:
vir, swir = composites.sel(band="nir"), composites.sel(band="swir16")
# NDVI is (B8-B4)/(B8+B4) https://sentinel.esa.int/web/sentinel/technical-guides/sentinel-2-msi/level-2a/algorithm
# NDSI is (B3-B11)/(B3+B11) https://sentinel.esa.int/web/sentinel/technical-guides/sentinel-2-msi/level-2a/algorithm
ndsi = (vir-swir)/(vir+swir)


In [ ]:
cleaned = rgb.ffill("time")[1:]

In [ ]:
%%time
gif_img = geogif.dgif(cleaned, fps=5).compute()

In [ ]:
# we turned ~7GiB of data into a 4MB GIF!
dask.utils.format_bytes(len(gif_img.data))

In [ ]:
gif_img

In [ ]:
cleaned = ndsi.ffill("time")[1:]

In [ ]:
%%time
gif_img = geogif.dgif(cleaned, fps=5).compute()

In [ ]:
# we turned ~7GiB of data into a 4MB GIF!
dask.utils.format_bytes(len(gif_img.data))

In [ ]:
gif_img